# Setup necessary libraries and Twitter access tokens

In [ ]:
#import tweepy
import numpy as np
import pandas as pd

In [ ]:
# input Twitter access tokens
access_token = 'paste your token here'
access_token_secret = 'paste your token secret here'
consumer_key = 'paste your consumer key here'
consumer_secret = 'paste your consumer secret here'

## Authenticate Twitter credentials

In [ ]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

## Pick a topic people are talking about on Twitter that you're interested in

In [ ]:
for tweet in api.search('health'):
    print(tweet.text)

## Create a dataframe to house Twitter data

In [ ]:
df = pd.DataFrame(columns = ['Tweets', 'User', 'User_statuses_count', 
                             'User_followers', 'User_location', 'User_verified', 
                             'fav_count', 'rt_count', 'tweet_date'])

In [ ]:
# NOTE: since we put our api.search into tweepy.Cursor, 
# it will not just stop at the first 100 tweets. 
# It will instead keep going on forever; that's why we are using 
# i as a counter to stop the loop after 1000 iterations.

def stream(data,file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'User_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i+=1
        if i == 20000:
            break
        else:
            pass

## Remember to update the date segment of the filename before running the next line of code!

In [ ]:
stream(data = ['health'], file_name = 'Health_tweets_YYMMDD')

## Optional: import previous Twitter data from Excel
Getting 20,000 tweets from Twitter takes TIME. If the very latest data isn't absolutely necessary and you have created the dataframe to house Twitter data previously, upload an existing file instead. See below.

In [ ]:
# Use an existing file. The file "Health_tweets_20200122.xlsx" is one I've created previously and is in the same directory as the Jupyter notebook.
df = pd.read_excel("Health_tweets_20200122.xlsx")

## Familiarising with data in existing file

In [ ]:
df.head()

In [ ]:
df.info()

## Let's analyse some Tweets

In [ ]:
from textblob import TextBlob
from wordcloud import WordCloud, STOPWORDS
import re
import chart_studio.plotly as py 
import plotly.graph_objects as go
from plotly.offline import iplot
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl', offline=True)

In [ ]:
# remove capitalisation etc from Tweets
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

## Create custom functions to analyse for polarity and subjectivity

In [ ]:
def label_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [ ]:
#def analyse_subjectivity(tweet):
#    analysis.sentiment.subjectivity
def analyse_polarity(clean_tweet):
    try:
        return TextBlob(clean_tweet).sentiment.polarity
    except:
        return None

In [ ]:
def analyse_subjectivity(clean_tweet):
    try:
        return TextBlob(clean_tweet).sentiment.subjectivity 
    except:
        return None

## Add sentiment calculations to dataframe

In [ ]:
# using 2 different methods to achieve same result to compare syntax of lambda & apply  
df['clean_tweet'] = df['Tweets'].apply(lambda x: clean_tweet(x))
df['Sentiment'] = df['clean_tweet'].apply(lambda x: label_sentiment(x))
df['Tweet_polarity'] = df['clean_tweet'].apply(analyse_polarity)
df['Tweet_subjectivity'] = df['clean_tweet'].apply(analyse_subjectivity)

## Eyeball samples of original Tweet versus cleaned Tweet and sentiment scores

In [ ]:
n = 20
print('Original tweet:\n'+ df['Tweets'][n])
print()
print('Clean tweet:\n'+ df['clean_tweet'][n])
print()
print('Sentiment:\n'+ df['Sentiment'][n])

In [ ]:
n = 147
print('Original tweet: '+ df['Tweets'][n])
print()
print('Clean tweet: '+ df['clean_tweet'][n])
print()
print('Sentiment: '+ df['Sentiment'][n])

## Save a copy of the latest output to Excel

In [ ]:
df.to_excel("Health_tweets_YYYYMMDD_w_sent_scores.xlsx")

## Visualising Twitter data

In [ ]:
df=pd.read_excel("Health_tweets_20200129_w_sent_scores.xlsx")
df.columns

# add viz of popular tweets?

In [ ]:
import matplotlib. pyplot as plt
import numpy as np

%matplotlib inline

In [ ]:
df.columns

In [ ]:
# using Matplotlib - data labels are not on hover
x = df['Tweet_polarity']
y = df['Tweet_subjectivity']
plt.scatter(x, y, alpha=0.2)
plt.xlabel("Tweet polarity (-1 = extremely negative, 1 = extremely positive)")
plt.ylabel("Tweet subjectivity (0 = very objective, 1 = very subjective)")
plt.title("Sentiment of 20,000 tweets that mention health")
plt.show()